<a href="https://colab.research.google.com/github/Rino04/Predicting-Premier-League-Matches/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import time

In [ ]:
# Intialize the global variables
years=list(range(2022,2020,-1))
all_matches =[]
standing_url ="https://fbref.com/en/comps/9/Premier-League_Stats"

In [ ]:
for year in years:
  data = requests.get(standing_url)
  soup = BeautifulSoup(data.text)
  standing_table = soup.select('table.stats_table')[0]

  links =[l.get("href") for l in standing_table.find_all('a')]
  links = [l for l in links if l and '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standing_url =f"https://fbref.com{previous_season}"


  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


    soup = BeautifulSoup(data.text)
    links =[l.get("href") for l in  soup.find_all('a')]
    links = [l for l in links if l and '/all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()


    try:
      team_data = matches.merge(shooting[["Date", "Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
    except ValueError:
      continue

    team_data = team_data[team_data['Comp'] == "Premier league"]
    team_data['Season'] = year
    team_data["team"] = team_name
    all_matches.append(team_data)
    time.sleep(1)




In [ ]:
match_df=pd.concat(all_matches)

In [ ]:
match_df.columns =[c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")

In [ ]:
data_df=pd.read_csv('matches.csv')
data_df.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
